
1. https://youtu.be/bFfWbQoVmIA?si=2MCtE9L9-fbn2k-o
2. https://youtu.be/tOMjTCO0htA?si=UXOw2ndTshKUx2n7

- Choose two most frequent bigrams
- add the most frequent  bigram to the vocabulary as a new token
- replace every bigram occurrence with the new token

In [32]:
text = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new "


In [38]:
from collections import Counter
from tqdm.auto import trange, tqdm

class BPETokenizer(object):
    def __init__(self, corpus):
        self.corpus = corpus
        self.end_of_word_token = "#"
        self.vocabulary = list()
        
        
        self.__prepare_corpus()
        self.__init_vocab()
        

    def __prepare_corpus(self):
        text = self.corpus
        text = text.split(" ")
        text = [t + self.end_of_word_token for t in text]
        text = "".join([t for t in text])
        self.corpus = text

    def __init_vocab(self):
        all_characters = list(self.corpus)
        self.vocabulary = list(set(all_characters))

    def make_bigrams(self):
        all_characters = list(self.corpus)
        character_bigrams = []

        for i in range(len(all_characters)):
            # if at the end of the list
            # break
            if i + 1 == len(all_characters):
                break
            else:
                current = all_characters[i]
                next_char = all_characters[i + 1]
        
                bigram = current + next_char
                character_bigrams.append(bigram)

        return character_bigrams
        

    def get_bigram_statistics(self):
        all_bigrams = self.make_bigrams()
        bigram_counts = Counter(all_bigrams)
        return bigram_counts

    def fit(self, k):
        pass
        
        
        
tokenizer = BPETokenizer(corpus=text)
tokenizer.get_bigram_statistics()

Counter({'er': 9,
         'r#': 9,
         'we': 8,
         '#n': 8,
         'ne': 8,
         'ew': 8,
         'lo': 7,
         'ow': 7,
         'w#': 7,
         '#l': 6,
         '#w': 3,
         'wi': 3,
         'id': 3,
         'de': 3,
         'es': 2,
         'st': 2,
         't#': 2,
         '##': 1})